In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [3]:
import os

In [12]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
from sklearn.model_selection import train_test_split

In [14]:
# Define the model architecture
class SSDModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(SSDModel, self).__init__()
        self.num_classes = num_classes

        # Backbone network (e.g. VGG16)
        self.backbone = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

        # Feature extractor layers
        self.feature_extractor = tf.keras.Sequential([
            tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D((2, 2))
        ])

        # SSD layers
        self.ssd_layers = []
        for i in range(6):  # 6 default boxes per feature map
            self.ssd_layers.append(SSDLayer(4 + i, num_classes))

        # Prediction layers
        self.prediction_layers = []
        for i in range(6):  # 6 feature maps
            self.prediction_layers.append(PredictionLayer(num_classes))

    def call(self, inputs):
        # Backbone network
        x = self.backbone(inputs)

        # Feature extractor layers
        x = self.feature_extractor(x)

        # SSD layers
        feature_maps = []
        for i, layer in enumerate(self.ssd_layers):
            x, feature_map = layer(x)
            feature_maps.append(feature_map)

        # Prediction layers
        predictions = []
        for i, layer in enumerate(self.prediction_layers):
            predictions.append(layer(feature_maps[i]))

        # Concatenate predictions
        predictions = tf.concat(predictions, axis=1)

        return predictions

class SSDLayer(tf.keras.layers.Layer):
    def __init__(self, num_defaults, num_classes):
        super(SSDLayer, self).__init__()
        self.num_defaults = num_defaults
        self.num_classes = num_classes

    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.num_defaults * (4 + self.num_classes), (3, 3), activation='linear', padding='same')

    def call(self, inputs):
        x = self.conv(inputs)
        x = tf.reshape(x, (-1, x.shape[1], x.shape[2], self.num_defaults, 4 + self.num_classes))
        return x, x[:, :, :, :, :4]

class PredictionLayer(tf.keras.layers.Layer):
    def __init__(self, num_classes):
        super(PredictionLayer, self).__init__()
        self.num_classes = num_classes

    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(self.num_classes, (3, 3), activation='softmax', padding='same')

    def call(self, inputs):
        x = self.conv(inputs)
        return x

def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (64, 64))  # Update resizing to (64, 64)
    return

def get_label(path):
    file_name = tf.strings.split(path, '/')[-1]
    if'non-vehicle' in file_name:
        return 1
    else:
        return 2

# Create the model
model = SSDModel(num_classes=2)  # 2 classes: vehicle and non-vehicle

# Compile the model
model.compile(optimizer='adam', loss='ssd_loss', metrics=['accuracy'])

# Load the dataset (replace with your dataset path)
dataset_path = '/content/drive/MyDrive/vehicle_dataset'

# Create a dataset object
dataset = tf.data.Dataset.list_files(dataset_path + '/*/*.png')
dataset = dataset.map(lambda x: (load_image(x), get_label(x)))

# Define the size of the training set
train_size = int(0.8 * dataset.cardinality())

# Split the dataset into training and testing subsets
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# Train the model
model.fit(train_dataset, epochs=10)

TypeError: in user code:

    File "<ipython-input-14-b4a25cc69f87>", line 100, in None  *
        lambda x: (load_image(x), get_label(x))
    File "<ipython-input-14-b4a25cc69f87>", line 84, in get_label  *
        if'non-vehicle' in file_name:

    TypeError: argument of type 'SymbolicTensor' is not iterable
